In [1]:
import requests
import dotenv
import os
import json

dotenv.load_dotenv()
token = os.getenv("TMDB_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}"
}

url = "https://api.themoviedb.org/3/genre/movie/list?language=fr"
response = requests.get(url, headers=headers)

genres_str = response.json()
id_to_genre = {genre['id']: genre['name'] for genre in genres_str['genres']}

page = 1
list_movies = []
while page <= 20:
    url = f"https://api.themoviedb.org/3/movie/top_rated?language=fr-FR&page={page}"
    response = requests.get(url, headers=headers)
    response = response.json()
    
    for i in range(0, len(response["results"])):
        data = {
            "genres": [id_to_genre[i] for i in response["results"][i]["genre_ids"]],
            "poster": f"https://image.tmdb.org/t/p/original{response['results'][i].get('poster_path')}"
        }
        list_movies.append(data)
    page += 1
    
with open("movies.json", "w") as f:
    json.dump(list_movies, f, indent=4)

In [2]:
# 16 = Animation & 27 = Horreur

import requests
import json
import dotenv

dotenv.load_dotenv()
api_key = os.getenv("TMDB_TOKEN")

genres = {"16": "Animation", "27": "Horreur"}

params = {
    'language': 'fr-FR',
    'include_adult': False,
    'include_video': False,
    'vote_count.gte': 100,
    "sort_by": "popularity.desc"
}

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# URL de l'endpoint de découverte de films
url = 'https://api.themoviedb.org/3/discover/movie'

all_movies = []
for genre_id, genre_name in genres.items():
    for i in range(1, 31):
        
        params["with_genres"] = genre_id
        params["page"] = i
        
        response = requests.get(url, params=params, headers=headers)
        if response.status_code != 200:
            print(f"Erreur {response.status_code} : {response.text}")
            break
        
        data = response.json()
        
        for movie in data["results"]:
            custom_data = {"genre" : genre_name, "poster": f"https://image.tmdb.org/t/p/original{movie.get('poster_path')}"}
            all_movies.append(custom_data)

with open("movies_binary_classif.json", "w") as dataset:
    json.dump(all_movies, dataset)